In [3]:
import librosa
import soundfile
import os, glob, pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [4]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result   

In [5]:
#Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [6]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("E:\Speech_Emotion_Rcognition\Dataset\Actor_*\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [7]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

C:\Users\Tuhin\AppData\Local\Temp\ipykernel_8916\3263455183.py:16: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\Tuhin\AppData\Local\Temp\ipykernel_8916\3263455183.py:16: FutureWarning: Pass y=[0.0000000e+00 3.0517578e-05 3.0517578e-05 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\Tuhin\AppData\Local\Temp\ipykernel_8916\3263455183.py:16: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00 -3.0517578e-05 ... -3.0517578e-05
 -3.0517578e-05 -3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users

In [8]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [9]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [10]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(activation='relu',alpha=0.01, batch_size=400, beta_1=0.9,beta_2=0.999,early_stopping=False, epsilon=1e-08, hidden_layer_sizes=100, learning_rate='adaptive', learning_rate_init=0.001, max_fun=15000, max_iter=800, momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5, random_state=None, shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False, warm_start=False)

In [11]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=400, hidden_layer_sizes=100,
              learning_rate='adaptive', max_iter=800)

In [12]:
#Predict for the test set
y_pred=model.predict(x_test)
# print(y_pred)

In [13]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 70.31%


In [14]:
df=pd.DataFrame(columns=['Actual','Predicted'])
df['Actual']=y_test
df['Predicted']=y_pred
# df.to_csv('E:\Speech_Emotion_Rcognition\Dataset\Result.csv',index=False)

#Save the model to disk

In [15]:
with open('modelForPrediction.pkl','wb') as f:
    pickle.dump(model,f)

In [21]:
filename = 'modelForPrediction.pkl'
loaded_model = pickle.load(open(filename, 'rb'))
feature=extract_feature('E:/dnld/Record (online-voice-recorder.com).wav', mfcc=True, chroma=True, mel=True)
feature=feature.reshape(1,-1)
print(loaded_model.predict(feature))
# Loaded models accuracy
live_accuracy=accuracy_score(y_true=y_test, y_pred=loaded_model.predict(x_test))
print("Accuracy: {:.2f}%".format(live_accuracy*100))
result = loaded_model.score(x_test, y_test)
print(result * 100)

['fearful']
Accuracy: 70.31%
70.3125


C:\Users\Tuhin\AppData\Local\Temp\ipykernel_8916\3263455183.py:16: FutureWarning: Pass y=[0.         0.         0.         ... 0.00030518 0.00048828 0.00012207] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
